<a href="https://colab.research.google.com/github/elijahmoon-hub/Elijah-s-repository/blob/main/ElijMoonUnit3Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd

In [5]:
data = pd.read_csv("https://raw.githubusercontent.com/elijahmoon-hub/Elijah-s-repository/refs/heads/main/Unit3/HistoricalData_1759340440919.csv")
data

,Date,Close/Last,Volume,Open,High,Low
0,09/30/2025,666.18,86288030,662.93,666.6500,661.610
1,09/29/2025,663.68,73499020,664.36,665.2800,661.860
2,09/26/2025,661.82,69179210,659.51,662.3700,657.880
3,09/25/2025,658.05,89622070,657.94,659.4056,654.405
4,09/24/2025,661.10,68082230,664.51,664.6100,659.670
...,...,...,...,...,...,...
181,01/08/2025,589.49,47304670,588.70,590.5799,585.195
182,01/07/2025,588.63,60393050,597.42,597.7500,586.780
183,01/06/2025,595.36,47679440,596.27,599.7000,593.600
184,01/03/2025,591.95,37888460,587.53,592.6000,586.430
